In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_val_predict, ShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [17]:
contador = 0

head = ['0', 'word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d', 
        'word_freq_our', 'word_freq_over', 'word_freq_remove', 'word_freq_internet', 'word_freq_order', 
        'word_freq_mail', 'word_freq_receive', 'word_freq_will', 'word_freq_people', 'word_freq_report', 
        'word_freq_addresses', 'word_freq_free', 'word_freq_business', 'word_freq_email', 'word_freq_you', 
        'word_freq_credit', 'word_freq_your', 'word_freq_font', 'word_freq_000', 'word_freq_money',
        'word_freq_hp', 'word_freq_hpl', 'word_freq_george', 'word_freq_650', 'word_freq_lab', 
        'word_freq_labs', 'word_freq_telnet', 'word_freq_857', 'word_freq_data', 'word_freq_415', 
        'word_freq_85', 'word_freq_technology', 'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 
        'word_freq_direct', 'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project', 
        'word_freq_re', 'word_freq_edu', 'word_freq_table', 'word_freq_conference', 'char_freq_;', 
        'char_freq_(', 'char_freq_[', 'char_freq_!', 'char_freq_$', 'char_freq_#', 
        'capital_run_length_average', 'capital_run_length_longest', 'capital_run_length_total', 'spam']

# lendo todos os atributos do arquivo
df = pd.read_table('spambase.data', delimiter=None, header=None, names=head)
df.head()

,0,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,"0,0.64,0.64,0,0.32,0,0,0,0,0,0,0.64,0,0,0,0.32...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"0.21,0.28,0.5,0,0.14,0.28,0.21,0.07,0,0.94,0.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"0.06,0,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,0....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Eliminando NaNs
for item in head:
    if item != '0': 
        df[item] = ''

df.head()

,0,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,"0,0.64,0.64,0,0.32,0,0,0,0,0,0,0.64,0,0,0,0.32...",,,,,,,,,,...,,,,,,,,,,
1,"0.21,0.28,0.5,0,0.14,0.28,0.21,0.07,0,0.94,0.2...",,,,,,,,,,...,,,,,,,,,,
2,"0.06,0,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,0....",,,,,,,,,,...,,,,,,,,,,
3,"0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0...",,,,,,,,,,...,,,,,,,,,,
4,"0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0...",,,,,,,,,,...,,,,,,,,,,


In [12]:
listaColunas = ['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d', 'word_freq_our', 
                'word_freq_over', 'word_freq_remove', 'word_freq_internet', 'word_freq_order', 
                'word_freq_mail', 'word_freq_receive', 'word_freq_will', 'word_freq_people', 
                'word_freq_report', 'word_freq_addresses', 'word_freq_free', 'word_freq_business',
                'word_freq_email', 'word_freq_you', 'word_freq_credit', 'word_freq_your', 
                'word_freq_font', 'word_freq_000', 'word_freq_money', 'word_freq_hp', 'word_freq_hpl', 
                'word_freq_george', 'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word_freq_telnet',
                'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85', 'word_freq_technology', 
                'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct', 
                'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project', 
                'word_freq_re', 'word_freq_edu', 'word_freq_table', 'word_freq_conference', 
                'char_freq_;', 'char_freq_(', 'char_freq_[', 'char_freq_!', 'char_freq_$', 
                'char_freq_#', 'capital_run_length_average', 'capital_run_length_longest', 
                'capital_run_length_total', 'spam']

df[listaColunas] = df['0'].str.split(',', expand=True)

listaColunasDeletar = ['0', 'word_freq_our', 'word_freq_over', 'word_freq_email', 'word_freq_addresses',
                       'word_freq_you', 'word_freq_your', 'word_freq_000', 'word_freq_hp', 'word_freq_hpl', 
                       'word_freq_george', 'word_freq_650', 'word_freq_lab', 'word_freq_labs', 
                       'word_freq_telnet', 'word_freq_857', 'word_freq_415', 'word_freq_85', 
                       'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct', 
                       'word_freq_cs', 'word_freq_original', 'word_freq_re', 'word_freq_table', 
                       'char_freq_;', 'char_freq_(', 'char_freq_[', 'capital_run_length_average', 
                       'capital_run_length_longest', 'capital_run_length_total']

df = df.drop(listaColunasDeletar, axis=1)
        
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,...,word_freq_data,word_freq_technology,word_freq_meeting,word_freq_project,word_freq_edu,word_freq_conference,char_freq_!,char_freq_$,char_freq_#,spam
0,0,0.64,0.64,0,0,0,0,0,0,0.64,...,0,0,0,0,0,0,0.778,0,0,1
1,0.21,0.28,0.5,0,0.21,0.07,0,0.94,0.21,0.79,...,0,0,0,0,0,0,0.372,0.18,0.048,1
2,0.06,0,0.71,0,0.19,0.12,0.64,0.25,0.38,0.45,...,0,0,0,0,0.06,0,0.276,0.184,0.01,1
3,0,0,0,0,0.31,0.63,0.31,0.63,0.31,0.31,...,0,0,0,0,0,0,0.137,0,0,1
4,0,0,0,0,0.31,0.63,0.31,0.63,0.31,0.31,...,0,0,0,0,0,0,0.135,0,0,1


In [14]:
rotulos = df['spam']
print(rotulos)

0       1
1       1
2       1
3       1
4       1
       ..
4596    0
4597    0
4598    0
4599    0
4600    0
Name: spam, Length: 4601, dtype: object


In [16]:
caracteristicas = df.drop(['spam'], axis=1)
caracteristicas.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,...,word_freq_money,word_freq_data,word_freq_technology,word_freq_meeting,word_freq_project,word_freq_edu,word_freq_conference,char_freq_!,char_freq_$,char_freq_#
0,0,0.64,0.64,0,0,0,0,0,0,0.64,...,0,0,0,0,0,0,0,0.778,0,0
1,0.21,0.28,0.5,0,0.21,0.07,0,0.94,0.21,0.79,...,0.43,0,0,0,0,0,0,0.372,0.18,0.048
2,0.06,0,0.71,0,0.19,0.12,0.64,0.25,0.38,0.45,...,0.06,0,0,0,0,0.06,0,0.276,0.184,0.01
3,0,0,0,0,0.31,0.63,0.31,0.63,0.31,0.31,...,0,0,0,0,0,0,0,0.137,0,0
4,0,0,0,0,0.31,0.63,0.31,0.63,0.31,0.31,...,0,0,0,0,0,0,0,0.135,0,0


# Perceptron Multicamadas

In [ ]:
def relatorioClassificaçãoPontuaçãoPrecisao(y_true, y_pred):
    contador = contador + 1
    print("Sequencia: "+ str(contador))
    print("classification_report(y_true, y_pred)")
    print(classification_report(y_true, y_pred))
    print("\n")
    return accuracy_score(y_true, y_pred) 

In [ ]:
contador = 0
classificadorMLP = MLPClassifier(hidden_layer_sizes=(57, 57, 57))
predicao = cross_val_predict(classificadorMLP, caracteristicas, rotulos, cv=10)
print(predicao)

In [ ]:
score = cross_val_score(classificadorMLP, 
                        caracteristicas, 
                        rotulos, cv=10, 
                        scoring=make_scorer(relatorioClassificaçãoPontuaçãoPrecisao))

print("Perceptron Multicamadas Impressão de todos os scores dos testes")
print(score)
print("\n")

print("Perceptron Multicamadas score medio: ", str(score.mean() * 100) + ' %')
print("Perceptron Multicamadas Acurácia (precisão média): ", str(accuracy_score(rotulos, predicao) * 100) + ' %')
print("\n")

In [ ]:
matrizConfusao = confusion_matrix(rotulos, predicao)

print("Matriz de Confusão Perceptron Multicamadas")
pd.crosstab(rotulos, predicao, rownames=['Rotulo'], colnames=['Predição'], margins=True)

In [ ]:
plt.matshow(matrizConfusao)
plt.ylabel('Rotulo')
plt.xlabel('Predição')
plt.title('Perceptron Multicamadas Matriz de Confusão')
plt.colorbar()
plt.show()

# Função Base Radial

In [ ]:
classificadorFBR = GaussianProcessClassifier(kernel=1.0 * RBF(1.0)).fit(caracteristicas, rotulos)  # funcao para fbr
predicao = classificadorFBR.predict(caracteristicas)
print(predicao)

In [ ]:
validacaoCruzada = ShuffleSplit(n_splits=10, test_size=0.1)
score = cross_val_score(classificadorFBR, caracteristicas, rotulos, cv=validacaoCruzada)

print("Função Base Radial Impressão de todos os scores dos testes")
print(score)
print("\n")

print("Função Base Radial score medio: ", str(score.mean() * 100) + ' %')
print("Função Base Radial Acurácia (precisão média): ", str(accuracy_score(rotulos, predicao) * 100) + ' %')
print("\n")

In [ ]:
print('Matriz de Confusão Função Base Radial')
pd.crosstab(rotulos, predicao, rownames=['Conhecido'], colnames=['Perdição'], margins=True)

In [ ]:
matrizConfusao = confusion_matrix(rotulos, predicao)
plt.matshow(matrizConfusao)
plt.ylabel('Rotulo')
plt.xlabel('Perdição')
plt.title('Função Base Radial Matriz de Confusão')
plt.colorbar()
plt.show()